# Initilalizing

### Connect to Drive

In [ ]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
%cd /gdrive/MyDrive/
%ls

/gdrive/MyDrive
'Colab Notebooks'/                       IncpetionModel1/
 Dataset/                                model.png
 FinalModel/                             S3CRET/
 Gabriele/                               SimpleLSTMForecasting/
 IncpetionFtAugModel1/                   SimplerModel/
 IncpetionFtAugWeighModel1/              StandardModel/
 IncpetionFtAugWeighModel1_more_layer/   training/
 IncpetionFtModel1/


In [ ]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
import math
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

2.7.0


### Set seed for reproducibility

In [ ]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Load the Data

In [ ]:
dataset = pd.read_csv('Dataset/Training.csv')
print(dataset.shape)
dataset.head()

(68528, 7)


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,7.97698,4.33494,10.67282,1.76692,3.22440,51.681460,3.65434
1,8.07824,4.44616,10.56160,1.70716,3.32566,51.563598,3.47672
2,8.02844,4.22372,10.56160,1.64906,3.17460,50.863080,3.47672
3,8.02844,4.22372,10.56160,1.70716,3.17460,45.841581,3.47672
4,7.87572,4.44616,10.45038,1.70716,3.27586,47.126421,3.47672


## Normalize

In [ ]:
# Normalize 
dataset_min = dataset.min()
dataset_max = dataset.max()
print(dataset_min)
print(dataset_max)
dataset_norm = (dataset-dataset_min)/(dataset_max-dataset_min)
dataset_norm.head()

Sponginess            -0.972080
Wonder level          -8.450380
Crunchiness          -37.581721
Loudness on impact    -9.028060
Meme creativity       -1.275860
Soap slipperiness     -6.006860
Hype root            -42.769221
dtype: float64
Sponginess            15.106680
Wonder level          18.122899
Crunchiness           41.138801
Loudness on impact    14.035980
Meme creativity        6.056360
Soap slipperiness     77.371620
Hype root             31.024420
dtype: float64


,Sponginess,Wonder level,Crunchiness,Loudness on impact,Meme creativity,Soap slipperiness,Hype root
0,0.556576,0.481134,0.612986,0.468044,0.613765,0.691885,0.629100
1,0.562874,0.485320,0.611573,0.465453,0.627575,0.690471,0.626693
2,0.559777,0.476949,0.611573,0.462934,0.606973,0.682070,0.626693
3,0.559777,0.476949,0.611573,0.465453,0.606973,0.621844,0.626693
4,0.550279,0.485320,0.610160,0.465453,0.620783,0.637254,0.626693


### Windowing 


In [ ]:
TELESCOPE = 864
WINDOWS_LENGTH = 2000
VALIDATION_SPLIT = 0.2

In [ ]:
future = dataset_norm[-WINDOWS_LENGTH:]
future = np.expand_dims(future, axis=0)
future.shape

(1, 2000, 7)

In [ ]:
def build_sequences(df, target_labels, window=2000, stride=20, telescope=864): #1156/4 = 288 time-steps into the future
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float32')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float32')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

In [ ]:
target_labels = dataset_norm.columns

In [ ]:
X_train, y_train = build_sequences(dataset_norm, target_labels, window =WINDOWS_LENGTH, telescope = TELESCOPE)
X_val = X_train[-math.floor(len(X_train)*VALIDATION_SPLIT):,:,:]
y_val = y_train[-math.floor(len(y_train)*VALIDATION_SPLIT):,:,:]
X_train = X_train[0:len(X_train)-math.floor(len(X_train)*VALIDATION_SPLIT), : , :] 
y_train = y_train[0:len(y_train)-math.floor(len(y_train)*VALIDATION_SPLIT), : , :]


In [ ]:
X_train.shape, y_train.shape, X_val.shape, y_val.shape

((2686, 2000, 7), (2686, 864, 7), (671, 2000, 7), (671, 864, 7))

In [ ]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
    plt.show()

# LSTM bidirectional

![link text](https://i.stack.imgur.com/GmKNd.png)

In [ ]:
multi_lstm_bid_model = tf.keras.Sequential([
    # Shape [batch, time, features] => [batch, lstm_units].
    # Adding more `lstm_units` just overfits more quickly.
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32, return_sequences=False)),
    # Shape => [batch, out_steps*features].
    tf.keras.layers.Dense(TELESCOPE*7,
                          kernel_initializer=tf.initializers.zeros()),
    # Shape => [batch, out_steps, features].
    tf.keras.layers.Reshape([TELESCOPE, 7])
])

multi_lstm_bid_model.compile(loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.metrics.MeanAbsoluteError(), tf.keras.metrics.RootMeanSquaredError()])

In [ ]:
#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
multi_lstm_bid_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_bid=multi_lstm_bid_model.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

Epoch 1/200
42/42 [==============================] - 24s 471ms/step - loss: 0.0908 - root_mean_squared_error: 0.3014 - val_loss: 0.0245 - val_root_mean_squared_error: 0.1564 - lr: 0.0010
Epoch 2/200
42/42 [==============================] - 18s 439ms/step - loss: 0.0199 - root_mean_squared_error: 0.1412 - val_loss: 0.0215 - val_root_mean_squared_error: 0.1468 - lr: 0.0010
Epoch 3/200
42/42 [==============================] - 18s 434ms/step - loss: 0.0191 - root_mean_squared_error: 0.1381 - val_loss: 0.0214 - val_root_mean_squared_error: 0.1462 - lr: 0.0010
Epoch 4/200
42/42 [==============================] - 18s 431ms/step - loss: 0.0184 - root_mean_squared_error: 0.1356 - val_loss: 0.0204 - val_root_mean_squared_error: 0.1429 - lr: 0.0010
Epoch 5/200
42/42 [==============================] - 18s 430ms/step - loss: 0.0166 - root_mean_squared_error: 0.1288 - val_loss: 0.0194 - val_root_mean_squared_error: 0.1392 - lr: 0.0010
Epoch 6/200
42/42 [==============================] - 18s 423ms/st

even this don't beat the simple LSTM

Both don't perform better than simple lstm

# LSTM autoencoder simpler model

![link text](https://editor.analyticsvidhya.com/uploads/83490ed.jpg)


In [ ]:
# E1D1
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(WINDOWS_LENGTH, 7))
encoder_l1 = tf.keras.layers.LSTM(32, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)

encoder_states1 = encoder_outputs1[1:]

#
decoder_inputs = tf.keras.layers.RepeatVector(TELESCOPE)(encoder_outputs1[0])

#
decoder_l1 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_outputs1 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(7))(decoder_l1)

#
model_e1d1 = tf.keras.models.Model(encoder_inputs,decoder_outputs1)

#
model_e1d1.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 2000, 7)]    0           []                               
                                                                                                  
 lstm_43 (LSTM)                 [(None, 32),         5120        ['input_3[0][0]']                
                                 (None, 32),                                                      
                                 (None, 32)]                                                      
                                                                                                  
 repeat_vector_15 (RepeatVector  (None, 864, 32)     0           ['lstm_43[0][0]']                
 )                                                                                          

In [ ]:
# E2D2
# n_features ==> no of features at each timestep in the data.
#
encoder_inputs = tf.keras.layers.Input(shape=(WINDOWS_LENGTH, 7))
encoder_l1 = tf.keras.layers.LSTM(32,return_sequences = True, return_state=True)
encoder_outputs1 = encoder_l1(encoder_inputs)
encoder_states1 = encoder_outputs1[1:]
encoder_l2 = tf.keras.layers.LSTM(32, return_state=True)
encoder_outputs2 = encoder_l2(encoder_outputs1[0])
encoder_states2 = encoder_outputs2[1:]
#
decoder_inputs = tf.keras.layers.RepeatVector(TELESCOPE)(encoder_outputs2[0])
#
decoder_l1 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_inputs,initial_state = encoder_states1)
decoder_l2 = tf.keras.layers.LSTM(32, return_sequences=True)(decoder_l1,initial_state = encoder_states2)
decoder_outputs2 = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(7))(decoder_l2)
#
model_e2d2 = tf.keras.models.Model(encoder_inputs,decoder_outputs2)
#
model_e2d2.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 2000, 7)]    0           []                               
                                                                                                  
 lstm_45 (LSTM)                 [(None, 2000, 32),   5120        ['input_4[0][0]']                
                                 (None, 32),                                                      
                                 (None, 32)]                                                      
                                                                                                  
 lstm_46 (LSTM)                 [(None, 32),         8320        ['lstm_45[0][0]']                
                                 (None, 32),                                                

In [ ]:
#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e1d1.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_e1d1=model_e1d1.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

Epoch 1/200
42/42 [==============================] - 20s 388ms/step - loss: 0.0134 - root_mean_squared_error: 0.1158 - val_loss: 0.0169 - val_root_mean_squared_error: 0.1300 - lr: 0.0010
Epoch 2/200
42/42 [==============================] - 15s 368ms/step - loss: 0.0127 - root_mean_squared_error: 0.1125 - val_loss: 0.0167 - val_root_mean_squared_error: 0.1292 - lr: 0.0010
Epoch 3/200
42/42 [==============================] - 15s 363ms/step - loss: 0.0124 - root_mean_squared_error: 0.1114 - val_loss: 0.0166 - val_root_mean_squared_error: 0.1290 - lr: 0.0010
Epoch 4/200
42/42 [==============================] - 15s 359ms/step - loss: 0.0122 - root_mean_squared_error: 0.1104 - val_loss: 0.0170 - val_root_mean_squared_error: 0.1305 - lr: 0.0010
Epoch 5/200
42/42 [==============================] - 15s 361ms/step - loss: 0.0121 - root_mean_squared_error: 0.1098 - val_loss: 0.0175 - val_root_mean_squared_error: 0.1325 - lr: 0.0010
Epoch 6/200
42/42 [==============================] - 15s 364ms/st

In [ ]:
#reduce_lr = tf.keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.90 ** x)
model_e2d2.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_e2d2=model_e2d2.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

Epoch 1/200
42/42 [==============================] - 38s 731ms/step - loss: 0.0475 - root_mean_squared_error: 0.2179 - val_loss: 0.0244 - val_root_mean_squared_error: 0.1563 - lr: 0.0010
Epoch 2/200
42/42 [==============================] - 28s 670ms/step - loss: 0.0196 - root_mean_squared_error: 0.1401 - val_loss: 0.0221 - val_root_mean_squared_error: 0.1486 - lr: 0.0010
Epoch 3/200
42/42 [==============================] - 28s 669ms/step - loss: 0.0180 - root_mean_squared_error: 0.1341 - val_loss: 0.0207 - val_root_mean_squared_error: 0.1439 - lr: 0.0010
Epoch 4/200
42/42 [==============================] - 28s 665ms/step - loss: 0.0166 - root_mean_squared_error: 0.1288 - val_loss: 0.0195 - val_root_mean_squared_error: 0.1396 - lr: 0.0010
Epoch 5/200
42/42 [==============================] - 27s 654ms/step - loss: 0.0148 - root_mean_squared_error: 0.1215 - val_loss: 0.0195 - val_root_mean_squared_error: 0.1395 - lr: 0.0010
Epoch 6/200
42/42 [==============================] - 28s 678ms/st

# CONV LSTM

In [ ]:
def build_CONV_LSTM_model(input_shape, output_shape):
    # Build the neural network layer by layer
    input_layer = tfkl.Input(shape=input_shape, name='Input')

    convlstm = tfkl.Bidirectional(tfkl.LSTM(64, return_sequences=True))(input_layer)
    convlstm = tfkl.Conv1D(128, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.MaxPool1D()(convlstm)
    convlstm = tfkl.Bidirectional(tfkl.LSTM(128, return_sequences=True))(convlstm)
    convlstm = tfkl.Conv1D(256, 3, padding='same', activation='relu')(convlstm)
    convlstm = tfkl.GlobalAveragePooling1D()(convlstm)
    convlstm = tfkl.Dropout(.5)(convlstm)

    # In order to predict the next values for more than one channel,
    # we can use a Dense layer with a number given by telescope*num_channels,
    # followed by a Reshape layer to obtain a tensor of dimension 
    # [None, telescope, num_channels]
    dense = tfkl.Dense(output_shape[-1]*output_shape[-2], activation='relu')(convlstm)
    output_layer = tfkl.Reshape((output_shape[-2],output_shape[-1]))(dense)
    output_layer = tfkl.Conv1D(output_shape[-1], 1, padding='same')(output_layer)

    # Connect input and output through the Model class
    model = tfk.Model(inputs=input_layer, outputs=output_layer, name='model')

    # Compile the model
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.Adam(), metrics=['mae'])

    # Return the model
    return model

In [ ]:
conv_model = build_CONV_LSTM_model(input_shape, output_shape)
conv_model.summary()
tfk.utils.plot_model(conv_model, expand_nested=True)

NameError: ignored

In [ ]:
conv_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_conv=conv_model.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

#CNN LSTM Autoeconder



[documentation](https://towardsdatascience.com/cnn-lstm-based-models-for-multiple-parallel-input-and-multi-step-forecast-6fe2172f7668)



In [ ]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, Bidirectional, Input, Flatten, Activation, Reshape, RepeatVector, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

In [ ]:
model_enc_dec_cnn = Sequential()
model_enc_dec_cnn.add(Conv1D(filters=3, kernel_size=9, activation='relu', input_shape=(WINDOWS_LENGTH, 7)))
model_enc_dec_cnn.add(Conv1D(filters=3, kernel_size=11, activation='relu'))
model_enc_dec_cnn.add(MaxPooling1D(pool_size=2))
model_enc_dec_cnn.add(Flatten())
model_enc_dec_cnn.add(RepeatVector(TELESCOPE))
model_enc_dec_cnn.add(LSTM(200, activation='relu', return_sequences=True))
model_enc_dec_cnn.add(TimeDistributed(Dense(100, activation='relu')))
model_enc_dec_cnn.add(TimeDistributed(Dense(7)))


In [ ]:
model_enc_dec_cnn.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_conv_lstm=model_enc_dec_cnn.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

does not work

#Multi-Head CNN-LSTM Model

In [ ]:
input_layer = Input(shape=(WINDOWS_LENGTH, 7)) 
head_list = []
for i in range(0, 7):
 conv_layer_head = Conv1D(filters=4, kernel_size=7, activation='relu')(input_layer)
 conv_layer_head_2 = Conv1D(filters=6, kernel_size=11, activation='relu')(conv_layer_head)
 conv_layer_flatten = Flatten()(conv_layer_head_2)
 head_list.append(conv_layer_flatten)
 
concat_cnn = Concatenate(axis=1)(head_list)
reshape = Reshape((head_list[0].shape[1], 7))(concat_cnn)
lstm = LSTM(100, activation='relu')(reshape)
repeat = RepeatVector(TELESCOPE)(lstm)
lstm_2 = LSTM(100, activation='relu', return_sequences=True)(repeat)
dropout = Dropout(0.2)(lstm_2)
dense = Dense(7, activation='linear')(dropout)
multi_head_cnn_lstm_model = Model(inputs=input_layer, outputs=dense)


In [ ]:
multi_head_cnn_lstm_model.compile(optimizer=tf.keras.optimizers.Adam(), 
                    loss=tf.losses.MeanSquaredError(),
                      metrics=[tf.keras.metrics.RootMeanSquaredError()])
history_multi_conv_lstm=multi_head_cnn_lstm_model.fit(
    X_train,
    y_train,
    epochs=200,
    validation_data=(X_val,y_val),
    batch_size=64,
    verbose=1,
    callbacks=[
        tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True),
        #reduce_lr,
        tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-5)
    ]).history

too much time

# Attention